Fourth LangGraph with conditional edges

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

In [13]:
class State(TypedDict):
    number1 : int
    number2 : int 
    operation1 : str
    number3 : int
    number4 : int
    operation2 : str
    finalNumber1 : int
    finalNumber2 : int

In [14]:
def add_node(state : State) -> State:
    """node to add two numbers"""
    state["finalNumber1"] = state["number1"] + state["number2"]
    return state

def subtract_node(state : State) -> State:
    """node to add two numbers"""
    state["finalNumber1"] = state["number1"] - state["number2"]
    return state

def decide_node(state : State) -> State:
    """node to decide which node to take next"""
    if (state["operation1"] == "+"):
        return "addition_operation"
    else:
        return "subtraction_operation"
    
def add_node2(state : State) -> State:
    """node to add two numbers"""
    state["finalNumber2"] = state["number3"] + state["number4"]
    return state

def subtract_node2(state : State) -> State:
    """node to add two numbers"""
    state["finalNumber2"] = state["number3"] - state["number4"]
    return state

def decide_node2(state : State) -> State:
    """node to decide which node to take next"""
    if (state["operation2"] == "+"):
        return "addition_operation2"
    else:
        return "subtraction_operation2"    

In [15]:
graph = StateGraph(State)

graph.add_node("add_node", add_node)
graph.add_node("add_node2", add_node2)
graph.add_node("subtract_node", subtract_node)
graph.add_node("subtract_node2", subtract_node2)
graph.add_node("router", lambda state:state)
graph.add_node("router2", lambda state:state)

graph.add_edge(START, "router")
graph.add_conditional_edges(
    "router",
    decide_node,
    {
        "addition_operation" : "add_node",
        "subtraction_operation" : "subtract_node"
    }
)
graph.add_edge("add_node", "router2")
graph.add_edge("subtract_node", "router2")
graph.add_conditional_edges(
    "router2",
    decide_node2,
    {
        "addition_operation2" : "add_node2",
        "subtraction_operation2" : "subtract_node2"
    }
)
graph.add_edge("add_node2",END)
graph.add_edge("subtract_node2", END)

app = graph.compile()

In [16]:
app.get_graph().print_ascii()

             +-----------+                 
             | __start__ |                 
             +-----------+                 
                   *                       
                   *                       
                   *                       
              +--------+                   
              | router |                   
              +--------+                   
            ...         ...                
           .               .               
         ..                 ..             
 +----------+          +---------------+   
 | add_node |          | subtract_node |   
 +----------+          +---------------+   
            ***         ***                
               *       *                   
                **   **                    
              +---------+                  
              | router2 |                  
              +---------+                  
            ...          ..                
           .               ..   

In [18]:
input = State(number1=10, operation1="-", number2=5, number3=7, number4=2, operation2="+", finalNumber1=0, finalNumber2=0)
print(app.invoke(input))

{'number1': 10, 'number2': 5, 'operation1': '-', 'number3': 7, 'number4': 2, 'operation2': '+', 'finalNumber1': 5, 'finalNumber2': 9}
